# Bulding the Rydberg Hamiltonian

This notebook contains the code and data to generate the results for the first part of the paper. This includes only classical stuff.

Structure of the notebook:
- [Build the material structure](#build_material)
- [Build the test/test set](#test_test)
- [Write CRYSTAL input files](#write_input)
- [Read CRYSTAL output files](#read_output)
- [Mapping to the Rydberg Hamiltonian](#mapping_to_ryd)
- [Approximations](#approximations)
- [Compare to QUBO model](#QUBO)
- [Classical Monte Carlo](#monte_carlo)

In [2]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import shutil as sh

from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr

from scipy.optimize import minimize
from scipy import constants

k_b = constants.physical_constants['Boltzmann constant in eV/K'][0]

from pymatgen.core.structure import Structure, Molecule
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer, PointGroupAnalyzer
from pymatgen.io.ase import AseAtomsAdaptor

from ase.visualize import view

from utils import *
from random_structures import *
from QUBO_models import *

import json
def vview(structure):
    view(AseAtomsAdaptor().get_atoms(structure))

eV_to_rad_s = 1.519267447321156e15


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Build the Material Structure
<a id="build_material"></a>

This section will focus on building the material structure, including relevant parameters and visualizations.

In [3]:
lattice = np.array([[ 1.233862, -2.137112,  0.      ],
                   [ 1.233862,  2.137112,  0.      ],
                   [ 0.      ,  0.      ,  8.685038]])

graphene = Structure(lattice, species=['C','C'], coords=[[2/3, 1/3, 0. ],[1/3, 2/3, 0.]])
graphene = SpacegroupAnalyzer(graphene).get_conventional_standard_structure()

supercell_order = 2
scaling_matrix = np.array([[supercell_order, 0, 0],
                            [0, supercell_order, 0],
                            [0, 0, 1]])
graphene_scell = copy.deepcopy(graphene)
graphene_scell.make_supercell(scaling_matrix)
# Reorder the atoms in the supercell so they follow the convention we are using (top to bottom, left to right).
ordering = [1,5,0,3,4,7,2,6]
graphene_scell = Structure(graphene_scell.lattice,graphene_scell.atomic_numbers,
                           graphene_scell.frac_coords[ordering])

graphene_mol_r_6_6 = cut_graphene_rectangle(graphene,15,14,center=True)
graphene_mol_r_6_6.remove_sites([1,5,11,18,25,32])
vview(graphene_mol_r_6_6)
# graphene_mol_r_6_6.to_file('/Users/brunocamino/Desktop/UCL/rydberg_atoms/data/structures/graphene_mol_r_6_6.xyz')

Add periodic boundary conditions

In [4]:
cell = np.array([[16.04, -2.134, 0.0],
                [ 0.0, 12.8 ,0.0],
                [0.0, 0.0, 10.0]])
graphene_mol_r_6_6_pbc = Structure(cell, graphene_mol_r_6_6.atomic_numbers, 
                                   graphene_mol_r_6_6.cart_coords, coords_are_cartesian=True)

# vview(graphene_mol_r_6_6_pbc)
# graphene_mol_r_6_6_pbc.to_file('/Users/brunocamino/Desktop/UCL/rydberg_atoms/data/structures/graphene_mol_r_6_6.cif')

## Build the Test/test Set
<a id="test_test"></a>

In this section, we'll create the test and train datasets using the prepared material structure.

In [5]:
atom_indices = get_all_configurations(graphene_mol_r_6_6_pbc)

binary_an = []

for n,i in enumerate(np.arange(1,11,1)):
    
    active_sites = np.where(np.array(graphene_mol_r_6_6_pbc.atomic_numbers)==6)[0]
    N_atoms = i
    
    structures_random = generate_random_structures(graphene_mol_r_6_6_pbc,atom_indices=atom_indices,
                                                       N_atoms=N_atoms,
                                                       new_species=7,N_config=100,DFT_config=10,
                                                       return_multiplicity=False,
                                                       active_sites=active_sites)
    # print(i,len(structures_random))

    num_structures = len(structures_random)
    
    for structure in structures_random:
        binary_an_tmp = np.zeros(len(active_sites),dtype='int')
        binary_an_tmp[np.where(np.array(structure.atomic_numbers) == 7)[0]] = 1
        binary_an.append(binary_an_tmp)

binary_an = np.array(binary_an)

# np.savetxt('data/crystal/graphene/pbc/graphene_mol_r_6_6_pbc_index_test.csv',binary_an,delimiter=',',fmt='%d')

## Write CRYSTAL Input Files
<a id="write_input"></a>

Here, we will generate the input files for the CRYSTAL simulation using the defined structure and parameters.

### Train set

In [ ]:
# TRAIN
# Load the input template and indices
with open('data/crystal/graphene/pbc/graphene_mol_r_6_6_pbc.d12') as file:
    input_template_save = file.readlines()

N_indices = np.genfromtxt(
    'data/crystal/graphene/pbc/graphene_mol_r_6_6_pbc_index_train.csv',
    delimiter=','
).astype(int)

# Iterate over the training indices
for i, N_index in enumerate(N_indices):
    # Get positions and count of N atoms
    N_cry_position = np.where(N_index == 1)[0] + 1
    N_N = np.sum(N_index == 1)

    # Prepare the input template
    input_template = copy.deepcopy(input_template_save)
    input_template.insert(3, f'{len(N_cry_position)}\n')

    for j, N in enumerate(N_cry_position):
        input_template.insert(4 + j, f'{N} 7\n')

    # Define output file paths
    d12_filename = f'data/crystal/graphene/pbc/train_set/graphene_mol_r_6_6_h_{N_N}_{i % 10}.d12'
    gui_filename = f'data/crystal/graphene/pbc/train_set/graphene_mol_r_6_6_h_{N_N}_{i % 10}.gui'

    # ONLY RUN ONCE BECAUSE IT OVERWRITES
    # Write the modified template to the output file
    # with open(d12_filename, 'w') as file:
    #     file.writelines(input_template)

    # Copy the GUI file to the output directory
    # sh.copy(
    #     './data/crystal/graphene/pbc/graphene_mol_r_6_6_pbc.gui',
    #     gui_filename
    # )

    # Print the command for running the script
#  print(f'/work/e05/e05/bcamino/runCRYSTAL/Pcry_slurm_multi graphene_mol_r_6_6_h_{N_N}_{i % 10} &')

### Test set

In [ ]:
import copy
import shutil as sh
import numpy as np

# Test
# Load the input template and indices
with open('data/crystal/graphene/pbc/graphene_mol_r_6_6_pbc.d12') as file:
    input_template_save = file.readlines()

N_indices = np.genfromtxt(
    'data/crystal/graphene/pbc/graphene_mol_r_6_6_pbc_index_test.csv',
    delimiter=','
).astype(int)

# Iterate over the testing indices
for i, N_index in enumerate(N_indices):
    # Get positions and count of N atoms
    N_cry_position = np.where(N_index == 1)[0] + 1
    N_N = np.sum(N_index == 1)

    # Prepare the input template
    input_template = copy.deepcopy(input_template_save)
    input_template.insert(3, f'{len(N_cry_position)}\n')

    for j, N in enumerate(N_cry_position):
        input_template.insert(4 + j, f'{N} 7\n')

    # Define output file paths
    d12_filename = f'data/crystal/graphene/pbc/test_set/graphene_mol_r_6_6_h_{N_N}_{i % 10}.d12'
    gui_filename = f'data/crystal/graphene/pbc/test_set/graphene_mol_r_6_6_h_{N_N}_{i % 10}.gui'


    # ONLY RUN ONCE BECAUSE IT OVERWRITES
    # Write the modified template to the output file
    # with open(d12_filename, 'w') as file:
    #     file.writelines(input_template)

    # Run the following only to generate the input files, then use the
    # ones generated by the calculation
    
    # # Copy the GUI file to the output directory
    # sh.copy(
    #     './data/crystal/graphene/pbc/graphene_mol_r_6_6_pbc.gui',
    #     gui_filename
    # )

    # Print the command for running the script
#  print(f'/work/e05/e05/bcamino/runCRYSTAL/Pcry_slurm_multi graphene_mol_r_6_6_h_{N_N}_{i % 10} &')

## Read CRYSTAL Output Files
<a id="read_output"></a>

This section covers how to parse and interpret the output files produced by the CRYSTAL simulation.

In [8]:
E_N = extract_final_energy('data/crystal/reference_states/N2.out') / 2
E_graphene = extract_final_energy('data/crystal/graphene/pbc/graphene_mol_r_6_6_pbc.out')
E_C = E_graphene / 78

structures_train, energies_train, structures_test, energies_test = process_dataset(
    train_folder='data/crystal/graphene/pbc/train_set/',
    test_folder='data/crystal/graphene/pbc/test_set/',
    N_sites=78,
    E_N=E_N,
    E_C=E_C
)

## Mapping to the Rydberg Hamiltonian
<a id="mapping_to_ryd"></a>

We'll map the results from CRYSTAL simulations to the Rydberg Hamiltonian in this section.

In [9]:
neighbor_count = [3,6,3,3,2]
neighbour_distances = np.array([1,np.sqrt(3),2,np.sqrt(7),3])
scaling_factors = 1/np.power(neighbour_distances,6)

structure = read_gui_file('data/crystal/graphene/pbc/graphene_mol_r_6_6_pbc.gui')
structure = SpacegroupAnalyzer(structure).get_symmetrized_structure()
atom_indices_graphene = get_all_configurations(structure)
N_atom = 7

num_sites = structure.num_sites

X_train, y_train = build_test_train_set(structures_train,energies_train,atom_indices_graphene,N_atom)

X_test, y_test = build_test_train_set(structures_test,energies_test,atom_indices_graphene,N_atom)

reg_coef = build_H_ryd(structure, X_train, y_train, neighbor_count, scaling_factors)

y_pred = calculate_Ryd_energy_pbc(structure, X_test, reg_coef, neighbor_count, scaling_factors)
print(f'Reg coefficient 0 = {reg_coef[0]}, Reg coefficient 1 = {reg_coef[1]}')

R²: 0.000000
Reg coefficient 0 = 0.0, Reg coefficient 1 = 0.0


#### Periodic

In [10]:
# Calculate the predicted energies
y_pred = calculate_Ryd_energy_pbc(structure, X_test, reg_coef, neighbor_count, scaling_factors)

# Mean Squared Error
mse = mean_squared_error(y_test, y_pred)

# Percentage MSE relative to mean squared value
mse_percent = 100 * mse / np.mean(np.square(y_test))

print(f"Mean Squared Error (MSE): {mse:.6e}")
print(f"Percentage MSE (relative to ⟨y²⟩): {mse_percent:.2f}%")

# Spearman's rank correlation
rho, p_value = spearmanr(y_pred, y_test)
print(f"Spearman's Rank Correlation Coefficient (ρ): {rho:.4f}")
print(f"P-value: {p_value:.4e}")

Mean Squared Error (MSE): 5.159005e-06
Percentage MSE (relative to ⟨y²⟩): 100.00%
Spearman's Rank Correlation Coefficient (ρ): nan
P-value: nan


/var/folders/f2/8kc7y9697m59bwltxjd42y300000gn/T/ipykernel_30804/4126642277.py:14: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  rho, p_value = spearmanr(y_pred, y_test)


#### Molecule

In [11]:
from sklearn.metrics import mean_squared_error

# Calculate the predicted energies
y_pred = calculate_Ryd_energy_mol(structure, X_test, reg_coef, scaling_factors)

# Mean Squared Error
mse = mean_squared_error(y_test, y_pred)

# Percentage MSE relative to mean squared value
mse_percent = 100 * mse / np.mean(np.square(y_test))

print(f"Mean Squared Error (MSE): {mse:.6e}")
print(f"Percentage MSE (relative to ⟨y²⟩): {mse_percent:.2f}%")

# Spearman's rank correlation
rho, p_value = spearmanr(y_pred, y_test)
print(f"Spearman's Rank Correlation Coefficient (ρ): {rho:.4f}")
print(f"P-value: {p_value:.4e}")



Mean Squared Error (MSE): 5.159005e-06
Percentage MSE (relative to ⟨y²⟩): 100.00%
Spearman's Rank Correlation Coefficient (ρ): nan
P-value: nan


/var/folders/f2/8kc7y9697m59bwltxjd42y300000gn/T/ipykernel_30804/109325501.py:16: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  rho, p_value = spearmanr(y_pred, y_test)


### Energy to R

In [12]:
# Given values
C6 = 5.42e-24  # C6 constant in rad m^6 / s

coulomb_interaction = reg_coef[1] * eV_to_rad_s  # Calculate coulomb interaction

# Calculate R
R = (C6 / coulomb_interaction) ** (1 / 6)  # R in meters

# Convert R to micrometers (1 meter = 1e6 micrometers)
R_micrometers = R * 1e6

# Print R in micrometers
print(f"R in micrometers: {R_micrometers:.6f} µm")

R in micrometers: inf µm


/var/folders/f2/8kc7y9697m59bwltxjd42y300000gn/T/ipykernel_30804/2958513610.py:7: RuntimeWarning: divide by zero encountered in scalar divide
  R = (C6 / coulomb_interaction) ** (1 / 6)  # R in meters


# THE END